# CNN

**Reference:** https://github.com/vinayakumarr/Network-Intrusion-Detection

**Dataset:** http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html

In [1]:
import numpy as np
np.random.seed(69)  # for reproducibility

import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, MaxPooling1D, Flatten

from sklearn.preprocessing import Normalizer

In [2]:
DATA = '../data/'
MODELS = '../models/'
TEST = 0.3

In [3]:
from utils.preprocess import train_and_evaluate_model

def build_model(activation):
    model = Sequential()
    model.add(Conv1D(8, 1, activation=activation, input_shape=X_train.shape[1:]))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(5,3, activation=activation))
    model.add(MaxPooling1D(2))
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(50,activation="relu"))
    model.add(Dense(23, activation="softmax"))
    return model

In [4]:
traindata = pd.read_csv(DATA+'kddcup.data.csv')
traindata

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,Y
0,0,0,0,0,215,45076,0,0,0,0,...,0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0
1,0,0,0,0,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.00,0.0,0.00,0.0,0.0,0
2,0,0,0,0,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.00,0.0,0.00,0.0,0.0,0
3,0,0,0,0,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.00,0.0,0.00,0.0,0.0,0
4,0,0,0,0,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.00,0.0,0.00,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4898426,0,0,0,0,212,2288,0,0,0,0,...,255,1.0,0.0,0.33,0.05,0.0,0.01,0.0,0.0,0
4898427,0,0,0,0,219,236,0,0,0,0,...,255,1.0,0.0,0.25,0.05,0.0,0.01,0.0,0.0,0
4898428,0,0,0,0,218,3610,0,0,0,0,...,255,1.0,0.0,0.20,0.05,0.0,0.01,0.0,0.0,0
4898429,0,0,0,0,219,1234,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.0,0.01,0.0,0.0,0


In [5]:
from utils import preprocess

traindata, testdata=preprocess.preprocess(traindata)

In [6]:
X = traindata.drop(columns=['Y'])
Y = traindata['Y']
C = testdata['Y']
T = testdata.drop(columns=['Y'])

In [7]:
scaler = Normalizer().fit(X)
trainX = scaler.transform(X)
testT = scaler.transform(T)

In [8]:
y_train= Y
y_test= C

In [9]:
y_train

125433     0
2162168    3
64208      0
3587456    2
2702069    3
          ..
906713     3
1954805    3
555009     2
3311998    3
2713363    3
Name: Y, Length: 3428901, dtype: int64

In [10]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(trainX, (trainX.shape[0],trainX.shape[1],1))
X_test = np.reshape(testT, (testT.shape[0],testT.shape[1],1))

In [11]:
relu_model=build_model("relu")
tanh_model=build_model("tanh")

relu_model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd",metrics=['accuracy'])
tanh_model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd",metrics=['accuracy'])

In [12]:
# train
history,report=train_and_evaluate_model(relu_model,X_train,y_train,X_test,y_test, epochs=1,batch_size=1024)

2512/2512 [==============================] - 14s 4ms/step - loss: 0.1710 - accuracy: 0.9484 - val_loss: 0.0397 - val_accuracy: 0.9897
Evaluating Model: 
1436/1436 [==============================] - 3s 2ms/step - loss: 0.0398 - accuracy: 0.9897


In [13]:
print(report)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99    291835
           1       0.52      0.28      0.37     13724
           2       0.99      0.99      0.99    321605
           3       1.00      1.00      1.00    842366

    accuracy                           0.99   1469530
   macro avg       0.87      0.82      0.84   1469530
weighted avg       0.99      0.99      0.99   1469530



In [14]:
relu_model.save("../models/relu_cnn.hdf5")

In [15]:
tanh_hist,tanh_report=train_and_evaluate_model(tanh_model,X_train,y_train,X_test,y_test, epochs=1,batch_size=1024)

2512/2512 [==============================] - 11s 4ms/step - loss: 0.1353 - accuracy: 0.9668 - val_loss: 0.0307 - val_accuracy: 0.9921
Evaluating Model: 
1436/1436 [==============================] - 3s 2ms/step - loss: 0.0310 - accuracy: 0.9921


In [16]:
print(tanh_report)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99    291835
           1       0.82      0.28      0.42     13724
           2       0.98      1.00      0.99    321605
           3       1.00      1.00      1.00    842366

    accuracy                           0.99   1469530
   macro avg       0.95      0.82      0.85   1469530
weighted avg       0.99      0.99      0.99   1469530



In [17]:
tanh_model.save("../models/tanh_cnn.hdf5")